# DeepState - Experiments

- Using DeepState with gluonts to run some experiments
- Follow this installation guide: http://gluon-ts-staging.s3-accelerate.dualstack.amazonaws.com/PR-271/4/install.html

Important note! Some of these models are not yet released in version "0.3.3" which is currently the most recent version on pypi from where you install packages via `pip install`. Hence, you should follow **Install from Source Code** (which I did) **Install from Github**

### Deep State Space Models (Rangapuram et al., 2018)

**DeepState** combines state space models (SSM) with a recurrent neural network (RNN). The SSM is applied *locally* to the individual time series that is parametrized using a *global* representation in form of a RNN. The RNN is trained on the entire dataset. 

- [`DeepStateEstimator`](http://gluon-ts.mxnet.io/master/api/gluonts/gluonts.model.deepstate.html#gluonts.model.deepstate.DeepStateEstimator)


In [7]:
# if version does not include DeepState, uncomment the following:
!pip install git+https://github.com/awslabs/gluon-ts.git

import gluonts
print(gluonts.__version__)

# standard imports
import numpy as np
import pandas as pd

# vis imports 
%matplotlib inline
import matplotlib.pyplot as plt

# json
import json

# gluon data 
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports 
from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
# from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

  Cloning https://github.com/awslabs/gluon-ts.git to /tmp/pip-req-build-lc26403u
  Installing build dependencies ... done
  Running setup.py bdist_wheel for gluonts ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-tp8gto7e/wheels/09/ba/b9/c830628df4d977a9dfe661f6001cdb861fc3f2524bac0ee433
Successfully built gluonts
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
0.3.4.dev68+ga894aee


In [2]:
def deep_state(seed=42, data="m4_daily", epochs=25, batches=50):
    dataset = get_dataset(data)

    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
#         ctx=mx.gpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepStateEstimator(
        trainer=trainer,
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         past_length=48,
        use_feat_static_cat=True,
        cardinality=[cardinality],
    )

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed

    pprint(output)


In [ ]:
%%time
results = []

if __name__ == "__main__":
    for i in range(42, 45):
        print("Seed:", i)
        res = deep_state(data="m4_monthly", seed=i, epochs=50, batches=500)
        pprint(res)
        results.append(res)
    
results

### DeepState - use_feat_static=False

In [8]:
def deep_state_no_feat(seed=42, data="m4_daily", epochs=25, batches=50):
    dataset = get_dataset(data)

    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
#         ctx=mx.gpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepStateEstimator(
        trainer=trainer,
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         past_length=48,
#         use_feat_static_cat=True,
#         cardinality=[cardinality],
    )

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed

    pprint(output)


In [10]:
%%time
results = []

if __name__ == "__main__":
    for i in range(42, 45):
        print("Seed:", i)
        res = deep_state_no_feat(data="m4_monthly", seed=i, epochs=50, batches=1000)
        pprint(res)
        results.append(res)
    
results

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_monthly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/1000 [00:00<?, ?it/s]

Seed: 42


INFO:root:Number of parameters in DeepStateTrainingNetwork: 21510
100%|██████████| 1000/1000 [01:37<00:00, 10.26it/s, avg_epoch_loss=-.278]
INFO:root:Epoch[0] Elapsed time 97.444 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-0.278151
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 1000/1000 [01:35<00:00, 10.49it/s, avg_epoch_loss=-.0851] 
INFO:root:Epoch[1] Elapsed time 95.304 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=-0.085067
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 1000/1000 [01:36<00:00, 10.40it/s, avg_epoch_loss=-.249]
INFO:root:Epoch[2] Elapsed time 96.202 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=-0.249471
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 1000/1000 [01:35<00:00, 10.53it/s, avg_epoch_loss=-.408]
INFO:root:Epoch[3] Elapsed time 95.009 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=-0.408244
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 1000/1000 [01:35<00:00,

KeyboardInterrupt: 